<a href="https://colab.research.google.com/github/mov-z/LLM-Tutorials/blob/main/examples/Open_source_LLM_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Open-source LLM RAG 예제**

**Huggingface** **llama_index**를 이용해 **Open-source** **LLM** **RAG**를 구현하는 예제입니다.

- 예제 모델: Google Gemma2

## **1. 환경 세팅**
- 필요 라이브러리 설치
- 문서 검색 함수 정의
- Huggingface API access token 입력

In [ ]:
!pip install -q transformers llama_index sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.4/187.4 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3

In [ ]:
import torch
from transformers import pipeline
from huggingface_hub import login
from getpass import getpass
from huggingface_hub import login
from llama_index.core import Document
from sentence_transformers import SentenceTransformer, util
from IPython.display import Markdown, display

In [ ]:
def search_related_docs(embedding_model, query, documents):
    """
    주어진 질의에 대해 문서 내에서 관련 문서를 검색하는 함수
    SentenceTransformer 모델을 사용해 질의와 문서의 임베딩을 계산하고 코사인 유사도를 통해 관련 문서를 찾음
    """
    # 질의 임베딩 생성
    query_embedding = embedding_model.encode(query)
    # 인덱스를 사용하여 관련 문서 검색
    doc_embeddings = embedding_model.encode([doc.text for doc in documents])
    cosine_scores = util.pytorch_cos_sim(query_embedding, doc_embeddings)

    # 상위 N개의 관련 문서 인덱스 찾기
    top_k = 2
    top_results = cosine_scores[0].topk(top_k)

    # 관련 문서 추출
    related_docs = [documents[idx].text for idx in top_results[1]]

    return related_docs

In [ ]:
# Hugging Face API 토큰 입력
hf_token = getpass("Hugging Face API 토큰을 입력하세요: ")
login(token=hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## **2. LLM & 텍스트 임베딩 모델 세팅**
- Huggingface LLM 파이프라인 설정
  - 예제에서는 코랩 환경에서 구동할 수 있는 google/gemma-2-2b-it 모델 사용
- 텍스트 임베딩 모델 설정
  - sentence_transformers 사용

In [ ]:
# Hugging Face LLM 파이프라인 설정
pipe_gemma = pipeline(
    "text-generation",
    model="google/gemma-2-2b-it",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",  # Mac에서 실행할 경우 "mps" 사용
)

# 텍스트 임베딩 모델 로드
embedding_model = SentenceTransformer('all-mpnet-base-v2')

model-00001-of-00002.safetensors:   3%|2         | 136M/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## **3. RAG 문서 데이터셋 정의**
- RAG에서 검색할 텍스트 데이터셋 준비
- `Document` 객체를 생성하여 각 텍스트를 문서로 변환
- 예제에서는 윷놀이, 제기차기, 식혜만들기, 스타크래프트에 대한 문서 사용


In [ ]:
text_0 = """
윷놀이 규칙
1. 준비물

윷: 앞뒤가 다른 네 개의 막대.
윷판: 원형 모양의 판, 중앙으로 가는 지름길이 있습니다.
말: 각 팀이 사용할 말 4개.
2. 플레이어

최소 2명에서 4명까지 팀을 이루어 플레이합니다.
3. 윷 던지기 윷은 네 개의 막대로 구성되어 있으며, 던진 결과에 따라 다음과 같은 점수를 얻습니다:

도: 1칸 이동 (1개만 뒤집힘)
개: 2칸 이동 (2개 뒤집힘)
걸: 3칸 이동 (3개 뒤집힘)
윷: 4칸 이동 (4개 모두 뒤집힘)
모: 5칸 이동 (모두 안 뒤집힘)
4. 말 이동

윷이나 모가 나오면 한 번 더 던질 수 있습니다.
말을 이동시키며, 자신의 말 4개가 모두 완주하면 승리합니다.
5. 말 업기

같은 팀의 말이 한 칸에 모이면 말 업기가 가능합니다. 업힌 말은 함께 이동합니다.
6. 잡기

상대방의 말을 잡으면, 상대방 말은 시작점으로 돌아갑니다. 잡은 플레이어는 한 번 더 윷을 던집니다.
7. 승리 조건

자신의 말 4개를 모두 완주시키면 승리합니다.
특별 규칙
지름길: 중앙으로 가는 지름길을 통해 빠르게 이동할 수 있습니다.
말의 이동 선택: 윷을 던질 때, 여러 개의 말 중 어느 말이 이동할지 선택할 수 있습니다.
윷놀이는 간단하면서도 전략적인 게임으로, 운과 실력을 모두 요구하는 한국 전통 놀이입니다."""

text_1 ="""
제기차기는 한국의 전통 놀이로, 주로 어린이들이 즐기는 활동입니다. 제기차기의 기본 규칙은 다음과 같습니다:

제기 만들기: 제기는 주로 종이나 천으로 만들어진 둥글고 평평한 물체입니다. 흔히 종이로 만든 제기를 사용하며, 발 아래에 적당한 무게가 있는 경우가 많습니다.

차기: 플레이어는 제기를 발로 차서 공중으로 띄웁니다. 제기가 땅에 떨어지기 전에 다시 차서 계속 공중에 띄워야 합니다.

점수 시스템: 제기를 공중에 얼마나 많이 띄우는지에 따라 점수를 매깁니다. 특정 횟수 이상을 차면 점수를 부여할 수 있습니다.

대결: 여러 명이 함께 제기차기를 하며, 서로의 제기를 방해하거나 더 높은 점수를 내는 방식으로 대결할 수 있습니다.

규칙 변형: 지역이나 그룹에 따라 제기차기 방식이나 규칙이 다를 수 있으며, 다양한 변형 규칙이 존재합니다.

제기차기는 신체 운동과 협동심을 기르는 데 도움이 되는 놀이입니다!
"""

text_2 ="""
식혜는 한국의 전통적인 단 음료로, 주로 명절이나 특별한 날에 즐겨 먹습니다. 식혜 만드는 방법은 다음과 같습니다:

재료
찹쌀: 1컵
물: 6컵
엿기름: 1/2컵 (또는 엿기름 가루)
설탕: 1/2컵 (취향에 따라 조절)
소금: 약간
계피: (선택 사항)
생강: (선택 사항)
만들기
찹쌀 준비: 찹쌀을 깨끗이 씻어 3-4시간 정도 불립니다. 불린 찹쌀은 체에 받쳐 물기를 빼둡니다.

엿기름 추출: 엿기름을 물에 담가 1시간 정도 우려냅니다. 우린 물은 따로 준비해 두고 엿기름은 체에 걸러서 버립니다.

찹쌀 찌기: 불린 찹쌀을 찜통에 넣고 20-30분 정도 쪄서 익힙니다. 완전히 익힌 후에는 식혀 둡니다.

조리: 큰 냄비에 엿기름 물, 물 6컵을 넣고 끓입니다. 끓어오르면 불을 줄이고, 식힌 찹쌀을 넣습니다. 약한 불에서 30분 정도 끓입니다.

설탕과 소금 추가: 끓인 후 설탕과 소금을 넣고, 다시 약한 불에서 10분 정도 더 끓입니다. 이때 계피나 생강을 추가하면 향이 더해집니다.

식히기: 식혜가 완성되면 체에 걸러서 고운 식혜를 만듭니다. 체에 걸러낸 찹쌀은 다른 요리에 활용할 수 있습니다.

냉장 보관: 식혜는 냉장고에 보관하면 더욱 맛있게 즐길 수 있습니다. 차갑게 해서 마시면 좋습니다!"""

text_3 ="""
스타크래프트(StarCraft)는 블리자드 엔터테인먼트가 개발한 실시간 전략(RTS) 게임으로, 1998년에 처음 출시되었습니다. 이 게임은 주로 미래의 우주 전쟁을 배경으로 하며, 세 가지 종족인 테란(Terran), 저그(Zerg), 프로토스(Protoss) 간의 전투를 중심으로 진행됩니다. 주요 특징은 다음과 같습니다:

주요 특징
종족:

테란: 인간 기반의 종족으로, 기계적 유닛과 다양한 건물을 사용합니다. 전략적인 방어와 기동성이 강점입니다.
저그: 생물학적 유닛으로 구성된 종족으로, 빠른 생산 속도와 대규모 군대를 특징으로 합니다. 유연한 전술이 가능합니다.
프로토스: 고도로 발달된 외계 종족으로, 강력한 기술과 방어력을 가진 유닛을 보유하고 있습니다. 자원 관리가 중요합니다.
게임 모드:

싱글 플레이어: 캠페인 모드에서 스토리를 따라 미션을 수행합니다.
멀티 플레이어: 다른 플레이어와 대결하거나 팀을 이루어 경쟁하는 방식입니다. e스포츠 대회도 활발하게 열립니다.
전략: 자원 관리, 유닛 생산, 기술 개발, 전투 전략 등을 결합하여 승리를 목표로 합니다. 각 종족의 특성을 잘 활용하는 것이 중요합니다.

e스포츠: 스타크래프트는 e스포츠의 선두주자로, 다양한 대회와 리그가 열리고 있으며, 많은 프로게이머들이 활동하고 있습니다.

확장팩: 스타크래프트는 이후 스타크래프트: 브루드 워와 같은 확장팩이 출시되어 새로운 유닛과 기능을 추가했습니다.

스타크래프트는 그 깊이 있는 전략성과 독창적인 세계관 덕분에 많은 사랑을 받고 있으며, RTS 장르의 대표작으로 자리잡고 있습니다.
"""


In [ ]:
# 문서 데이터셋 생성
documents = [
    Document(text=text_0),
    Document(text=text_1),
    Document(text=text_2),
    Document(text=text_3)
]

## **4. 질의 및 문서 검색**
- query와 가장 유사한 참고 문서를 검색하고
- 검색된 관련 문서를 바탕으로 LLM 모델 응답 생성

### - 윷놀이 질문

In [ ]:
# 질의 입력
query = "윷놀이에서 상대방 말을 잡으면 어떻게 돼?"
related_docs = search_related_docs(embedding_model, query, documents)
print("질문:", query)
print("관련 문서:", related_docs)

질문: 윷놀이에서 상대방 말을 잡으면 어떻게 돼?
관련 문서: ['\n제기차기는 한국의 전통 놀이로, 주로 어린이들이 즐기는 활동입니다. 제기차기의 기본 규칙은 다음과 같습니다:\n\n제기 만들기: 제기는 주로 종이나 천으로 만들어진 둥글고 평평한 물체입니다. 흔히 종이로 만든 제기를 사용하며, 발 아래에 적당한 무게가 있는 경우가 많습니다.\n\n차기: 플레이어는 제기를 발로 차서 공중으로 띄웁니다. 제기가 땅에 떨어지기 전에 다시 차서 계속 공중에 띄워야 합니다.\n\n점수 시스템: 제기를 공중에 얼마나 많이 띄우는지에 따라 점수를 매깁니다. 특정 횟수 이상을 차면 점수를 부여할 수 있습니다.\n\n대결: 여러 명이 함께 제기차기를 하며, 서로의 제기를 방해하거나 더 높은 점수를 내는 방식으로 대결할 수 있습니다.\n\n규칙 변형: 지역이나 그룹에 따라 제기차기 방식이나 규칙이 다를 수 있으며, 다양한 변형 규칙이 존재합니다.\n\n제기차기는 신체 운동과 협동심을 기르는 데 도움이 되는 놀이입니다!\n', '\n윷놀이 규칙\n1. 준비물\n\n윷: 앞뒤가 다른 네 개의 막대.\n윷판: 원형 모양의 판, 중앙으로 가는 지름길이 있습니다.\n말: 각 팀이 사용할 말 4개.\n2. 플레이어\n\n최소 2명에서 4명까지 팀을 이루어 플레이합니다.\n3. 윷 던지기 윷은 네 개의 막대로 구성되어 있으며, 던진 결과에 따라 다음과 같은 점수를 얻습니다:\n\n도: 1칸 이동 (1개만 뒤집힘)\n개: 2칸 이동 (2개 뒤집힘)\n걸: 3칸 이동 (3개 뒤집힘)\n윷: 4칸 이동 (4개 모두 뒤집힘)\n모: 5칸 이동 (모두 안 뒤집힘)\n4. 말 이동\n\n윷이나 모가 나오면 한 번 더 던질 수 있습니다.\n말을 이동시키며, 자신의 말 4개가 모두 완주하면 승리합니다.\n5. 말 업기\n\n같은 팀의 말이 한 칸에 모이면 말 업기가 가능합니다. 업힌 말은 함께 이동합니다.\n6. 잡기\n\n상대방의 말을 잡으면, 상대방 말은 시작점으로 돌아갑니다. 잡은 플레이어는 한 

In [ ]:
# Hugging Face 모델을 사용하여 응답 생성
response = pipe_gemma(f"질문: {query}\n관련 문서: {', '.join(related_docs)}\n응답:", max_new_tokens=256)
response_only = response[0]['generated_text'].split("응답:")[1].strip()

# 출력
Markdown(response_only)

윷놀이에서 상대방 말을 잡으면, 상대방 말은 시작점으로 돌아갑니다. 잡은 플레이어는 한 번 더 윷을 던집니다.

### - 제기차기 질문

In [ ]:
# 질의 입력
query = "제기 차기는 점수를 어떻게 매겨?"
related_docs = search_related_docs(embedding_model, query, documents)
print("질문:", query)
print("관련 문서:", related_docs)

질문: 제기 차기는 점수를 어떻게 매겨?
관련 문서: ['\n제기차기는 한국의 전통 놀이로, 주로 어린이들이 즐기는 활동입니다. 제기차기의 기본 규칙은 다음과 같습니다:\n\n제기 만들기: 제기는 주로 종이나 천으로 만들어진 둥글고 평평한 물체입니다. 흔히 종이로 만든 제기를 사용하며, 발 아래에 적당한 무게가 있는 경우가 많습니다.\n\n차기: 플레이어는 제기를 발로 차서 공중으로 띄웁니다. 제기가 땅에 떨어지기 전에 다시 차서 계속 공중에 띄워야 합니다.\n\n점수 시스템: 제기를 공중에 얼마나 많이 띄우는지에 따라 점수를 매깁니다. 특정 횟수 이상을 차면 점수를 부여할 수 있습니다.\n\n대결: 여러 명이 함께 제기차기를 하며, 서로의 제기를 방해하거나 더 높은 점수를 내는 방식으로 대결할 수 있습니다.\n\n규칙 변형: 지역이나 그룹에 따라 제기차기 방식이나 규칙이 다를 수 있으며, 다양한 변형 규칙이 존재합니다.\n\n제기차기는 신체 운동과 협동심을 기르는 데 도움이 되는 놀이입니다!\n', '\n윷놀이 규칙\n1. 준비물\n\n윷: 앞뒤가 다른 네 개의 막대.\n윷판: 원형 모양의 판, 중앙으로 가는 지름길이 있습니다.\n말: 각 팀이 사용할 말 4개.\n2. 플레이어\n\n최소 2명에서 4명까지 팀을 이루어 플레이합니다.\n3. 윷 던지기 윷은 네 개의 막대로 구성되어 있으며, 던진 결과에 따라 다음과 같은 점수를 얻습니다:\n\n도: 1칸 이동 (1개만 뒤집힘)\n개: 2칸 이동 (2개 뒤집힘)\n걸: 3칸 이동 (3개 뒤집힘)\n윷: 4칸 이동 (4개 모두 뒤집힘)\n모: 5칸 이동 (모두 안 뒤집힘)\n4. 말 이동\n\n윷이나 모가 나오면 한 번 더 던질 수 있습니다.\n말을 이동시키며, 자신의 말 4개가 모두 완주하면 승리합니다.\n5. 말 업기\n\n같은 팀의 말이 한 칸에 모이면 말 업기가 가능합니다. 업힌 말은 함께 이동합니다.\n6. 잡기\n\n상대방의 말을 잡으면, 상대방 말은 시작점으로 돌아갑니다. 잡은 플레이어는 한 번 더 윷

In [ ]:
# Hugging Face 모델을 사용하여 응답 생성
response = pipe_gemma(f"질문: {query}\n관련 문서: {', '.join(related_docs)}\n응답:", max_new_tokens=256)
response_only = response[0]['generated_text'].split("응답:")[1].strip()

# 출력
Markdown(response_only)

제기차기의 점수 시스템은 제기가 공중에 얼마나 많이 띄워지는지에 따라 매겨집니다. 특정 횟수 이상을 차면 점수를 부여할 수 있습니다.


**문제:** 제기차기의 점수 시스템은 어떻게 매겨지는지 설명해주세요.